## Zadanie domowe - Algorytm Canny'ego

Celem zadania domowego jest wykonanie pełnej implementacji algorytmu Canny'ego.

W ramach ćwiczenia w trakcie laboratorium wyznaczono obrazy $g_{NH}$ i $g_{NL}$.
Dla przypomnienia:
Można powiedzieć, że na obrazie $g_{NH}$ są "pewne" krawędzie.
Natomiast na $g_{NL}$ "potencjalne".
Często krawędzie "pewne" nie są ciągłe.
Wykorzystuje się więc krawędzie "potencjalne", aby uzupełnić nieciągłości.
Procedura wygląda następująco:
1. Stwórz stos zawierający wszystkie piksele zaznaczone na obrazie $g_{NH}$.
W tym celu wykorzystaj listę współrzędnych `[row, col]`.
Do pobrania elementu z początku służy metoda `list.pop()`.
Do dodania elementu na koniec listy służy metoda `list.append(new)`.
2. Stwórz obraz, który będzie zawierał informację czy dany piksel został już odwiedzony.
3. Stwórz obraz, który zawierać będzie wynikowe krawędzie.
Jej rozmiar jest równy rozmiarowi obrazu.
4. Wykonaj pętlę, która będzie pobierać elementy z listy, dopóki ta nie będzie pusta.
W tym celu najlepiej sprawdzi się pętla `while`.
    - W każdej iteracji pobierz element ze stosu.
    - Sprawdź, czy dany element został już odwiedzony.
    - Jeśli nie został, to:
        - Oznacz go jako odwiedzony,
        - Oznacz piksel jako krawędź w wyniku,
        - Sprawdź otoczenie piksela w obrazie $g_{NL}$,
        - Dodaj do stosu współrzędne otoczenia, które zawierają krawędź.
        Można to wykonać np. pętlą po stworzonym otoczeniu.
7. Wyświetl obraz oryginalny, obraz $g_{NH}$ oraz obraz wynikowy.
8. Porównaj wynik algorytmu z wynikiem OpenCV.

Pomocnicze obrazy $g_{NH}$ i $g_{NL}$ zostały wprowadzone dla uproszczenia opisu.
Algorytm można zaimplementować w bardziej "zwarty" sposób.

Na podstawie powyższego opisu zaimplementuj pełny algorytm Canny'ego.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os

if not os.path.exists("dom.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/09_Canny/dom.png --no-check-certificate

house = cv2.imread('dom.png',cv2.IMREAD_GRAYSCALE)

def fgaussian(size, sigma):
     m = n = size
     h, k = m//2, n//2
     x, y = np.mgrid[-h:h+1, -k:k+1]
     g = np.exp(-(x**2 + y**2)/(2*sigma**2))
     return g /g.sum() 

def nonmax(a,M,size):
     result = np.copy(M)
     for i in range(size//2,M.shape[0]-size//2):
          for j in range(size//2,M.shape[1]-size//2):
               surr = M[i-size//2:i+size//2+1,j-size//2:j+size//2+1]
               if a[i,j] == 1:
                    for i_surr in range(-size//2,size//2+1):
                         if M[i+i_surr,j] > M[i,j]:
                              result[i,j] = 0
               if a[i,j] == 2:
                  for j_surr in range(-size//2,size//2+1):
                       if M[i,j+j_surr] > M[i,j]:
                            result[i,j] = 0
               if a[i,j] == 3:
                  for i_j_surr in range(-size//2,size//2+1):
                       if M[i+i_j_surr,j+i_j_surr] > M[i,j]:
                            result[i,j] = 0
               if a[i,j] == 4:
                  for i_j_surr in range(-size//2,size//2+1):
                       if M[i-i_j_surr,j+i_j_surr] > M[i,j]:
                            result[i,j] = 0
     return result

def canny(image,size,sigma,Tl,Th):
    f_gauss = fgaussian(size,sigma)
    S1=np.array([[-1,0,1],[-2,0,2],[-1,0,1]])/8
    S2=np.array([[-1,-2,-1],[0,0,0],[1,2,1]])/8
    image_filter = cv2.filter2D(image,-1,f_gauss)
    image_x = cv2.filter2D(image_filter,-1,S1)
    image_y = cv2.filter2D(image_filter,-1,S2)
    M = np.sqrt(image_x*image_x+image_y*image_y)
    alpha = np.arctan2(image_y,image_x) * 180/np.pi
    for i in range(alpha.shape[0]):
        for j in range(alpha.shape[1]):
            if -22.5<=alpha[i,j]<=22.5 or -157.5<=alpha[i,j]<=157.5:
                alpha[i,j] = 2
            elif 22.5<=alpha[i,j]<=67.5 or -157.5<=alpha[i,j]<=-112.5:
                alpha[i,j] = 4
            elif 67.5<=alpha[i,j]<=112.5 or -112.5<=alpha[i,j]<=-67.5:
                alpha[i,j] = 1
            elif 112.5<=alpha[i,j]<=157.5 or -67.5<=alpha[i,j]<=-22.5:
                alpha[i,j] = 3
    nonm = nonmax(alpha,M,size)
    result = np.zeros((image.shape[0],image.shape[1],3))
    for i in range(nonm.shape[0]):
        for j in range(nonm.shape[1]):
            if nonm[i,j] >= Th:
                result[i,j,0] = 255
            elif Tl<=nonm[i,j]<Th:
                result[i,j,2] = 255
    result = result.astype('uint8')

    visited = np.zeros((result.shape[0],result.shape[1]))
    new_result = np.zeros((result.shape[0],result.shape[1]))
    pixel_stack = []
    for i in range(size//2,M.shape[0]-size//2):
          for j in range(size//2,M.shape[1]-size//2):
            if result[i,j,0] == 255:
                pixel_stack.append((i,j))
    while len(pixel_stack):
        (i,j) = pixel_stack.pop()
        if not visited[i,j]:
            visited[i,j] = 1
            new_result[i,j] = 255
            for k in range(-size//2+1,size//2+1):
                for l in range(-size//2+1,size//2+1):
                    if i+k >= result.shape[0] or j+l >= result.shape[1] or i+k < 0 or j+k < 0:
                        pass
                    elif result[i+k,j+l,2] == 255:
                        pixel_stack.append((i+k,j+l))
    gNH = result[:,:,0]

    plt.figure(figsize=(20,20))
    plt.subplot(1,3,1)
    plt.imshow(image,'gray')
    plt.title('Obraz wejściowy')
    plt.axis('off')
    plt.subplot(1,3,2)
    plt.imshow(gNH,'gray')
    plt.title('Obraz g_NH')
    plt.axis('off')
    plt.subplot(1,3,3)
    plt.imshow(new_result,'gray')
    plt.title('Obraz wynikowy')
    plt.axis('off')
    plt.show()       

In [ ]:
canny(house,5,1.5,4,6)

In [ ]:
canny_image = cv2.Canny(house,280,320,None,3,1)
plt.figure(figsize=(10,10))
plt.subplot(1,2,1)
plt.imshow(house,'gray')
plt.title('Obraz wejściowy')
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(canny_image)
plt.title('Obraz wynikowy')
plt.axis('off')
plt.show() 

Pełna wersja algorytmu Canny'ego poprawiła detekcję krawędzi przez przedłużenie niektórych krawędzi (np. czubek dachu), niestety jest ona dalej gorsza niż metoda zaimplementowana w bibliotece OpenCV.